<a href="https://colab.research.google.com/github/limnavonglouis/42-DSLR/blob/master/colab/notebooks/05_model_exploration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount("/content/drive")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
% cd /content/drive/My Drive/research/artexplorer

/content/drive/My Drive/research/artexplorer


# Load the model 

In [68]:
import torch
import torch.nn.functional as F
import torch.nn as nn
from torchvision import models, transforms

In [52]:
model_dir = 'model_output'

# n_classes =  len(dset_loaders['test'].dataset.classes)
n_classes = 51
model_resnet = models.resnet18(pretrained=True)
model_resnet.fc = nn.Sequential(
        nn.BatchNorm1d(512),
        nn.Linear(512, n_classes)
    )

model_resnet.load_state_dict(torch.load(model_dir+'/model.pth'))

<All keys matched successfully>

In [ ]:
model_resnet

# Model evaluation

In [6]:
import os 

In [7]:
artists_list = os.listdir('western_data/images')
artists_list.sort()

# Variables

In [96]:
from PIL import Image

In [9]:
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                                 std=[0.229, 0.224, 0.225])  

test_prep = transforms.Compose([
                transforms.CenterCrop(224), # default cropping 
                transforms.ToTensor(),
                normalize,
            ])

In [10]:
images_dir = 'test_western_data/images/'

# Functions

In [201]:
def display_image(image_name, dir):

  if dir: 
    image_path = images_dir + image_name
  else:
     image_path = image_name
  img_pil = Image.open(image_path)
  img_pil = img_pil.convert('RGB')
  img_pil = img_pil.resize((400, 400))
  display(img_pil)

def get_image(image_name):

  image_path = images_dir + image_name
  img_pil = Image.open(image_path)
  img_pil = img_pil.convert('RGB')
  # display(img_pil)
  img_tensor = test_prep(img_pil).float()
  img_variable = img_tensor.unsqueeze(0)

  return img_variable

In [225]:
def predict_artist(img_variable):
  
  model_resnet.eval()
  pred = model_resnet(img_variable)
  idx = torch.max(pred, 1)[1].numpy()

  return artists_list[idx[0]]

def view_artist(artist_name):
  for image_name in os.listdir(images_dir+artist_name)[:2]:
    display_image(artist_name+'/'+image_name, True)

In [13]:
def main(image_name):

  display_image(image_name, True)
  img_variable = get_image(image_name)
  artist_pred = predict_artist(img_variable)
  print(artist_pred)
  view_artist(artist_pred)
  

# Test

In [14]:
img_variable = get_image('Frida_Kahlo/Frida_Kahlo_5.jpg')

In [53]:
model_resnet.eval()
pred = model_resnet(img_variable)
#idx = torch.max(pred, 1)[1].numpy()
idxs = pred.detach().numpy().flatten().argsort()[-3:][::-1]
idxs

array([17, 39, 44])

# Result 1: Create interactive notebook

In [16]:
images_list = list()
for artist in os.listdir(images_dir):
  artist_images_list = [artist+'/'+el for el in os.listdir(images_dir+artist)]
  images_list.extend(artist_images_list)

In [17]:
from ipywidgets import interact

In [97]:
interact(main, image_name=images_list)

interactive(children=(Dropdown(description='image_name', options=('Paul_Gauguin/Paul_Gauguin_104.jpg', 'Paul_G…

<function __main__.main>

# Result 2: Activation map

In [35]:
import numpy as np

In [ ]:
import cv2
from IPython.display import Image as IPimage

In [19]:
finalconv_name = 'layer4'

In [59]:
features_blobs = []
def hook_save_feature(module, input, output):
    features_blobs.append(output.data.cpu().numpy())
model_resnet._modules.get(finalconv_name).register_forward_hook(hook_save_feature);

In [36]:
params = list(model_resnet.parameters())
weight_softmax = np.squeeze(params[-2].data.numpy())

In [38]:
def returnCAM(feature_conv, weight_softmax, class_idx):
    # generate the class activation maps upsample to 256x256
    size_upsample = (256, 256)
    bz, nc, h, w = feature_conv.shape  # (1, 512, 7, 7)
    output_cam = []
    for idx in class_idx:
        cam = weight_softmax[idx].dot(feature_conv.reshape((nc, h*w)))  # 49
        cam = cam.reshape(h, w)  # 7*7
        cam = cam - np.min(cam)
        cam_img = cam / np.max(cam)
        cam_img = np.uint8(255 * cam_img)
        output_cam.append(cv2.resize(cam_img, size_upsample))  # 256*256
    return output_cam

In [218]:
def get_prediction(image_name):

  img_variable = get_image(image_name)
  pred = model_resnet(img_variable)
  h_x = F.softmax(pred, dim=1).data.squeeze()
  probs, idx = h_x.sort(0, True)

  return probs.numpy(), idx.numpy()

# Activation map 

In [203]:
def display_cam(image_name, cam):

  path = images_dir + image_name
  img = cv2.imread(path)
  height, width, _ = img.shape
  heatmap = cv2.applyColorMap(cv2.resize(cam,(width, height)), cv2.COLORMAP_JET)
  result = heatmap * 0.3 + img * 0.5

  cv2.imwrite('CAM.jpg', result)
  display_image("CAM.jpg", False)


In [228]:
def main_cam(image_name):

  probs, idx = get_prediction(image_name)
  print(artists_list[idx[0]] + ' --> ' + str(probs[0]))
  print(artists_list[idx[1]] + ' --> ' + str(probs[1]))
  CAMs = returnCAM(features_blobs[-1], weight_softmax, [idx[0]])
  display_cam(image_name, CAMs[0])
  print(artists_list[idx[0]])
  view_artist(artists_list[idx[0]])

In [229]:
interact(main_cam, image_name=images_list)

interactive(children=(Dropdown(description='image_name', options=('Paul_Gauguin/Paul_Gauguin_104.jpg', 'Paul_G…

<function __main__.main_cam>